In [83]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 500)

import seaborn as sns

In [133]:
df = pd.read_json('data/data.json')

In [134]:
df = df.sample(frac=.3)

In [131]:
df.describe()

,approx_payout_date,body_length,channels,delivery_method,event_created,event_end,event_published,event_start,fb_published,gts,has_analytics,has_header,has_logo,name_length,num_order,num_payouts,object_id,org_facebook,org_twitter,sale_duration,sale_duration2,show_map,user_age,user_created,user_type,venue_latitude,venue_longitude
count,8.600000e+02,860.000000,860.000000,858.000000,8.600000e+02,8.600000e+02,8.510000e+02,8.600000e+02,860.000000,860.000000,860.000000,531.000000,860.000000,860.000000,860.000000,860.000000,8.600000e+02,856.000000,856.000000,848.000000,860.000000,860.000000,860.000000,8.600000e+02,860.000000,799.000000,799.000000
mean,1.351112e+09,3534.627907,6.229070,0.446387,1.346003e+09,1.350679e+09,1.341604e+09,1.350345e+09,0.111628,2537.939547,0.080233,0.193974,0.859302,42.029070,27.206977,35.260465,4.521893e+06,8.686916,4.522196,46.327830,50.137209,0.860465,383.718605,1.312849e+09,2.727907,33.900877,-56.161650
std,2.376931e+07,5100.296939,3.786176,0.611093,2.388310e+07,2.376929e+07,8.318260e+07,2.382789e+07,0.315091,9324.402647,0.271811,0.395781,0.347912,24.048367,87.310621,134.657085,1.964778e+06,12.413259,6.074930,49.077426,56.397453,0.346705,426.719394,4.166558e+07,1.100497,22.030312,71.200571
min,1.207375e+09,0.000000,0.000000,0.000000,1.205533e+09,1.206943e+09,0.000000e+00,1.206927e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.040110e+05,0.000000,0.000000,-1.000000,-1.000000,0.000000,0.000000,1.144878e+09,1.000000,-45.045208,-158.104286
25%,1.336256e+09,812.250000,5.000000,0.000000,1.331504e+09,1.335824e+09,1.331545e+09,1.335592e+09,0.000000,141.860000,0.000000,0.000000,1.000000,26.000000,3.000000,0.000000,3.045752e+06,0.000000,0.000000,16.000000,17.000000,1.000000,16.750000,1.285545e+09,1.000000,33.828270,-96.791242
50%,1.355540e+09,1936.000000,6.000000,0.000000,1.350977e+09,1.355108e+09,1.351037e+09,1.354988e+09,0.000000,472.115000,0.000000,0.000000,1.000000,38.000000,8.000000,2.000000,4.606588e+06,0.000000,0.000000,31.000000,35.000000,1.000000,239.000000,1.315568e+09,3.000000,39.965210,-79.399629
75%,1.370296e+09,4219.250000,8.000000,1.000000,1.365043e+09,1.369864e+09,1.365116e+09,1.369434e+09,0.000000,1495.132500,0.000000,0.000000,1.000000,53.000000,24.000000,15.000000,6.000009e+06,14.000000,11.000000,58.250000,62.250000,1.000000,618.000000,1.346811e+09,3.000000,44.091102,-3.166768
max,1.399032e+09,55128.000000,13.000000,3.000000,1.380815e+09,1.398600e+09,1.380815e+09,1.398571e+09,1.000000,163250.000000,1.000000,1.000000,1.000000,234.000000,1777.000000,1858.000000,8.608251e+06,91.000000,16.000000,408.000000,511.000000,1.000000,2393.000000,1.380815e+09,5.000000,56.423744,174.775236


In [35]:
df['acct_type'].unique()

array(['premium', 'tos_warn', 'fraudster_event', 'spammer_warn',
       'fraudster', 'locked', 'spammer_noinvite', 'spammer_limited',
       'tos_lock', 'fraudster_att'], dtype=object)

In [78]:
df_nums = df[['approx_payout_date', 'body_length', 'channels', 'delivery_method',
       'event_created', 'event_end', 'event_published', 'event_start',
       'fb_published', 'gts', 'has_analytics', 'has_header', 'has_logo',
       'name_length', 'num_order', 'num_payouts', 'object_id', 'org_facebook',
       'org_twitter', 'sale_duration', 'sale_duration2', 'show_map',
       'user_age', 'user_created', 'user_type', 'venue_latitude',
       'venue_longitude']]

In [232]:
def unique_col_indicator(df,col): 
    try:
        col_count = df.groupby(['user_created',col])['channels'].count().shape[0]
        user_count = df.groupby(['user_created'])['channels'].count().shape[0]
        if col_count == user_count:
            return True
        else:
           
            return False
    except:
        pass

In [231]:
def user_df():
    unique_cols = []
    for col in columns:
        if unique_col_indicator(df,col):
            unique_cols.append(col)
    user_unique_events = df.groupby('user_created').count().max(axis=1)
    user_unique_events = pd.DataFrame(user_unique_events,columns=['user_event_count'])
    return df[unique_cols].merge(user_unique_events,how='inner',on='user_created')

In [117]:
columns = ['acct_type', 'approx_payout_date', 'body_length', 'channels', 'country',
       'currency', 'delivery_method', 'description', 'email_domain',
       'event_created', 'event_end', 'event_published', 'event_start',
       'fb_published', 'gts', 'has_analytics', 'has_header', 'has_logo',
       'listed', 'name', 'name_length', 'num_order', 'num_payouts',
       'object_id', 'org_desc', 'org_facebook', 'org_name', 'org_twitter',
       'payee_name', 'payout_type', 'previous_payouts', 'sale_duration',
       'sale_duration2', 'show_map', 'ticket_types', 'user_age',
       'user_created', 'user_type', 'venue_address', 'venue_country',
       'venue_latitude', 'venue_longitude', 'venue_name', 'venue_state']


In [249]:
def user_unique_events_col():
    user_unique_events = df.groupby('user_created').count().max(axis=1)
    user_unique_events =  pd.DataFrame(user_unique_events,columns=['user_event_count'])
    return df.merge(user_unique_events,how='inner',on='user_created')

In [226]:
# user_df().merge(user_unique_events,on='user_created',how='inner')

In [224]:
user_unique_events = df.groupby('user_created').count().max(axis=1)
user_unique_events = pd.DataFrame(user_unique_events,columns=['user_event_count'])

In [182]:
def user_event_count():
    df['user_event_count'] = df.groupby('user_created').count().max(axis=1)
    return df

In [262]:
def drop_cols():
    df.drop(['acct_type'],axis=1)
    return df

In [256]:
def create_target():
    df['fraud'] = df['acct_type'].map({'fraudster_event': 1, 
                                   'premium': 0, 
                                   'spammer_warn': 0, 
                                   'fraudster': 1,
                                   'spammer_limited': 0, 
                                   'spammer_noinvite': 0,
                                   'locked': 1, 
                                   'tos_lock': 0,
                                   'tos_warn': 0,
                                   'fraudster_att': 1,
                                   'spammer_web': 0,
                                   'spammer': 0})
    return df

In [260]:
df = create_target()

In [267]:
num_columns = ['approx_payout_date', 'body_length', 'channels', 'delivery_method',
       'event_created', 'event_end', 'event_published', 'event_start',
       'fb_published', 'gts', 'has_analytics', 'has_header', 'has_logo',
       'name_length', 'num_order', 'num_payouts', 'object_id', 'org_facebook',
       'org_twitter', 'sale_duration', 'sale_duration2', 'show_map',
       'user_age', 'user_created', 'user_type', 'venue_latitude',
       'venue_longitude', 'user_event_count', 'user_unique_events']

target_column = ['fraud']

In [266]:
df[num_columns]

['approx_payout_date',
 'body_length',
 'channels',
 'delivery_method',
 'event_created',
 'event_end',
 'event_published',
 'event_start',
 'fb_published',
 'gts',
 'has_analytics',
 'has_header',
 'has_logo',
 'name_length',
 'num_order',
 'num_payouts',
 'object_id',
 'org_facebook',
 'org_twitter',
 'sale_duration',
 'sale_duration2',
 'show_map',
 'user_age',
 'user_created',
 'user_type',
 'venue_latitude',
 'venue_longitude',
 'user_event_count',
 'user_unique_events',
 'fraud']